In [1]:
from dataclasses import dataclass
from pathlib import Path
import joblib

import yaml
import pandas as pd
import os

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

from mlproject.exception import CustomException
import sys

from mlproject.constant import CONFIG_FILE_PATH, ML_MODEL_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH

from mlproject import logger
from mlproject.utils.common import create_directories, read_yaml, read_ml_model_yaml, save_bin
from box import ConfigBox 

In [2]:
%pwd

'c:\\Users\\ParagJadhav\\Downloads\\ML_Projects\\red_wine\\red_wine_quality_prediction\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\ParagJadhav\\Downloads\\ML_Projects\\red_wine\\red_wine_quality_prediction'

In [5]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    params: dict
    ml_models: object
    target_column: str

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH, #constant/__init__.py file gives the path of config.yaml file
        params_filepath = PARAMS_FILE_PATH, 
        schema_filepath = SCHEMA_FILE_PATH,
        ml_model_filepath = ML_MODEL_FILE_PATH): 
        
        self.config = read_yaml(config_filepath) #whatever is present inside the config.yaml file, it will read the it by providing the path of the config.yaml file.
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        self.ml_model = read_ml_model_yaml(ml_model_filepath)
            
        create_directories([self.config.artifacts_root]) #create the "artifacts" directory and name is present inside the config.yaml file.

        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        
        config = self.config.model_trainer
        schema =  self.schema.TARGET_COLUMN
        params = self.params
        ml_model = self.ml_model

        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            target_column = schema.name,
            params = params,
            ml_models = ml_model
            )

        return model_trainer_config

In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def data_spliting(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        self.X_train = train_data.drop([self.config.target_column], axis=1)
        self.X_test = test_data.drop([self.config.target_column], axis=1)
        self.y_train = train_data[[self.config.target_column]]
        self.y_test = test_data[[self.config.target_column]]


    def hyper_parameter_tunning(self):
        
        try:
            models= self.config.ml_models
            for key in models.keys():
                models[key] = globals()[models[key]]()

            params = self.config.params
            X_train = self.X_train
            y_train = self.y_train.values.ravel()
            report = {}

            for i in range(len(list(models))):
                
                model = list(models.values())[i]
                para = params[list(models.keys())[i]]
                
                gs = GridSearchCV(model,para,cv=3, n_jobs=-1)
                gs.fit(X_train,y_train)
                model.set_params(**gs.best_params_)
                model.fit(X_train,y_train)
                
                y_train_pred = model.predict(X_train)
                train_model_score = r2_score(y_train, y_train_pred)
                report[list(models.keys())[i]] = train_model_score
                best_model_score = max(sorted(report.values()))
                best_model_name = list(report.keys())[list(report.values()).index(best_model_score)]
                
                best_model = models[best_model_name]
            
            #print(report)
            if best_model_score<0.6:
                raise CustomException("No best model found")
                
            logger.info(f"Best found model on both training and testing dataset")
                
            save_bin(best_model, os.path.join(self.config.root_dir, self.config.model_name))

        except Exception as e:
            raise CustomException(e, sys)

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.data_spliting()
    model_trainer_config.hyper_parameter_tunning()
except Exception as e:
    raise CustomException(e, sys)

[2024-05-17 08:05:59,496: line_number-45: INFO: common: yaml file: config\config.yaml loaded successfully]


[2024-05-17 08:05:59,513: line_number-45: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-17 08:05:59,525: line_number-45: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-17 08:05:59,530: line_number-67: INFO: common: ml_model.yaml file: ml_model.yaml loaded successfully]
[2024-05-17 08:05:59,530: line_number-83: INFO: common: created directory at: artifacts]
[2024-05-17 08:05:59,540: line_number-83: INFO: common: created directory at: artifacts/model_trainer]
[2024-05-17 08:06:13,845: line_number-51: INFO: 116133265: Best found model on both training and testing dataset]
[2024-05-17 08:06:13,853: line_number-125: INFO: common: binary file saved at: artifacts/model_trainer\model.joblib]
